In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
engine = create_engine('sqlite:///tab.db')
conn = engine.connect()
conn
dfo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfo)))

9299 races loaded!


In [4]:
category = 'G'
dfo = dfo.loc[dfo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

3796 G races!


In [5]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head()

,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,results,runners
0,1,RICHMOND,NSW,C,G,2017-09-13,1,LADBROKES INFO HUB MDN HT1,2017-09-13 05:12:00,Paying,400,"[[4], [2], [1], [5]]",8.0,"[{""runnerName"": ""ZALI'S BLUEBOY"", ""runnerNumbe...","[[4], [2], [1], [5]]","[{'riderDriverName': '', 'fixedOdds': {'winDed..."
1,2,RICHMOND,NSW,C,G,2017-09-13,2,KEVIN WATERS TOWING MDN HT2,2017-09-13 05:32:00,Paying,400,"[[1], [8], [7], [2]]",8.0,"[{""runnerName"": ""SKETCHY LINDA"", ""runnerNumber...","[[1], [8], [7], [2]]","[{'riderDriverName': '', 'fixedOdds': {'winDed..."
2,3,RICHMOND,NSW,C,G,2017-09-13,4,PENRITHELECTRICAL.COM.AU STKS,2017-09-13 06:07:00,Paying,400,"[[9], [7], [8], [2]]",8.0,"[{""runnerName"": ""BROWZA BOB'"", ""runnerNumber"":...","[[9], [7], [8], [2]]","[{'riderDriverName': '', 'fixedOdds': {'winDed..."
3,4,RICHMOND,NSW,C,G,2017-09-13,5,RICHMONDGREYHOUNDS.COM.AU STKS,2017-09-13 06:27:00,Paying,618,"[[6], [7], [3], [1]]",8.0,"[{""runnerName"": ""MIAMI DEMON"", ""runnerNumber"":...","[[6], [7], [3], [1]]","[{'riderDriverName': '', 'fixedOdds': {'winDed..."
4,5,RICHMOND,NSW,C,G,2017-09-13,6,SWIFT FANCY @ STUD 1-2W STKS,2017-09-13 06:42:00,Paying,535,"[[6], [7], [1], [8]]",8.0,"[{""runnerName"": ""KILLING TIME"", ""runnerNumber""...","[[6], [7], [1], [8]]","[{'riderDriverName': '', 'fixedOdds': {'winDed..."


In [6]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
#     print(race)

    num_runners = len([r for r in race['runners'] if r['odds_win']])

    # add num_runners to all runners
    for runner in race['runners']:
#         print(runner)
        runner['num_runners'] = num_runners
        runner['race_type'] = race['race_type']

    data_all = data_all.append(race['runners'])
    
data_all.tail(10)

,barrierNumber,claimAmount,finishingPosition,fixedOdds,num_runners,odds_perc,odds_scale,odds_win,parimutuel,prediction,probability,race_type,rank_win,riderDriverName,runnerName,runnerNumber,trainerName
0,0,0,0,"{'winDeduction': 0, 'propositionNumber': 15135...",8,0.000000,0.000000,0.00,"{'bettingStatus': 'Scratched', 'returnWin': 0,...",0.000000,0.000000,G,10,,STYLE,9,JUSTIN WOOD
1,7,0,1,"{'winDeduction': 0, 'propositionNumber': 15136...",8,0.062500,0.051328,16.00,"{'bettingStatus': 'Normal', 'returnWin': 12.3,...",0.041421,0.040120,G,5,,SLEEPY AGAIN,10,GRAEME HALL
2,3,0,3,"{'winDeduction': 0, 'propositionNumber': 15135...",8,0.062500,0.051328,16.00,"{'bettingStatus': 'Normal', 'returnWin': 14.5,...",0.041421,0.040120,G,5,,LIL MISS CHEEKY,3,GRAEME HALL
3,4,0,0,"{'winDeduction': 0, 'propositionNumber': 15135...",8,0.062500,0.051328,16.00,"{'bettingStatus': 'Normal', 'returnWin': 20.4,...",0.041421,0.040120,G,5,,KING DEBELLO,4,DEREK COLDSTREAM
4,5,0,0,"{'winDeduction': 0, 'propositionNumber': 15135...",8,0.038462,0.031586,26.00,"{'bettingStatus': 'Normal', 'returnWin': 32.8,...",0.021799,0.021114,G,8,,MISSONI,5,MAY LEACH
5,6,0,0,"{'winDeduction': 0, 'propositionNumber': 15135...",8,0.270270,0.221959,3.70,"{'bettingStatus': 'Normal', 'returnWin': 4, 'r...",0.217899,0.211058,G,2,,WHO'S GOT STORM,6,TONY SCOTT
6,7,0,0,"{'winDeduction': 0, 'propositionNumber': 15135...",8,0.000000,0.000000,0.00,"{'bettingStatus': 'Scratched', 'returnWin': 0,...",0.000000,0.000000,G,10,,GOING TO JACKSON,7,BEN MACLEAN
7,8,0,2,"{'winDeduction': 0, 'propositionNumber': 15135...",8,0.066667,0.054750,15.00,"{'bettingStatus': 'Normal', 'returnWin': 8.5, ...",0.077599,0.075163,G,4,,NEW COURSE,8,JOHN RIGHETTI
8,1,0,0,"{'winDeduction': 0, 'propositionNumber': 15135...",8,0.571429,0.469284,1.75,"{'bettingStatus': 'Normal', 'returnWin': 1.9, ...",0.545415,0.528290,G,1,,JUNKYARD SARGE,1,JAMES JEFFRIES
9,2,0,4,"{'winDeduction': 0, 'propositionNumber': 15135...",8,0.083333,0.068437,12.00,"{'bettingStatus': 'Normal', 'returnWin': 13.6,...",0.045442,0.044015,G,3,,HARLAN ROCKS,2,KODY CHARLES


In [7]:
# drop scratched
data = data_all.dropna(subset=['odds_win'])
data = data[(data.odds_win > 0)]
data.describe()

,barrierNumber,claimAmount,finishingPosition,num_runners,odds_perc,odds_scale,odds_win,prediction,probability,rank_win,runnerNumber
count,29006.000000,29006.0,29006.000000,29006.000000,29006.000000,29006.000000,29006.000000,2.900600e+04,2.900600e+04,29006.000000,29006.000000
mean,4.393712,0.0,1.250052,7.720265,0.163178,0.130869,13.207495,1.283867e-01,1.308695e-01,4.276805,4.608150
std,2.273991,0.0,1.458515,0.728711,0.140159,0.109446,13.605410,1.229035e-01,1.228806e-01,2.227413,2.433648
min,0.000000,0.0,0.000000,4.000000,0.006623,0.003072,1.010000,2.216318e-14,1.629203e-14,1.000000,1.000000
25%,2.000000,0.0,0.000000,8.000000,0.058824,0.046382,4.400000,3.831820e-02,3.929290e-02,2.000000,3.000000
50%,4.000000,0.0,1.000000,8.000000,0.125000,0.098345,8.000000,9.642892e-02,9.780437e-02,4.000000,5.000000
75%,6.000000,0.0,2.000000,8.000000,0.227273,0.181155,17.000000,1.745844e-01,1.808891e-01,6.000000,7.000000
max,8.000000,0.0,4.000000,10.000000,0.990099,0.774723,151.000000,1.000000e+00,8.688590e-01,10.000000,10.000000


In [8]:
# get label data
Y = data['finishingPosition'] == 1
Y = Y.astype(int)
Y.describe()

count    29006.000000
mean         0.131076
std          0.337490
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: finishingPosition, dtype: float64

In [9]:
# xp = fixed odds perc
xp = data['odds_perc']
xp.describe()

count    29006.000000
mean         0.163178
std          0.140159
min          0.006623
25%          0.058824
50%          0.125000
75%          0.227273
max          0.990099
Name: odds_perc, dtype: float64

In [10]:
# xs = odds scaled
xs = data['odds_scale']
xs.describe()

count    29006.000000
mean         0.130869
std          0.109446
min          0.003072
25%          0.046382
50%          0.098345
75%          0.181155
max          0.774723
Name: odds_scale, dtype: float64

In [11]:
# xr = win rank
xr = data['rank_win']
xr.describe()

count    29006.000000
mean         4.276805
std          2.227413
min          1.000000
25%          2.000000
50%          4.000000
75%          6.000000
max         10.000000
Name: rank_win, dtype: float64

In [12]:
# xn = get num runners
xn = data['num_runners']
xn.describe()

count    29006.000000
mean         7.720265
std          0.728711
min          4.000000
25%          8.000000
50%          8.000000
75%          8.000000
max         10.000000
Name: num_runners, dtype: float64

In [13]:
X = pd.concat([xp, xs, xr, xn], axis=1)
X.describe()
# sns.pairplot(pd.concat([X, Y], axis=1))

,odds_perc,odds_scale,rank_win,num_runners
count,29006.000000,29006.000000,29006.000000,29006.000000
mean,0.163178,0.130869,4.276805,7.720265
std,0.140159,0.109446,2.227413,0.728711
min,0.006623,0.003072,1.000000,4.000000
25%,0.058824,0.046382,2.000000,8.000000
50%,0.125000,0.098345,4.000000,8.000000
75%,0.227273,0.181155,6.000000,8.000000
max,0.990099,0.774723,10.000000,10.000000


In [14]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 1000
print('epochs = {}'.format(epochs))

tag = '{}30x30'.format(category)
file_name = '/Users/jaco/code/tabby/models/{}.h5'.format(tag)

try:
    model = load_model(file_name)
except OSError:
    model = Sequential()
    model.add(Dense(30, activation='relu', input_dim=n))
    model.add(Dense(30, activation='relu'))
#     model.add(Dense(40, activation='relu'))
#     model.add(Dense(40, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

input dimension = 4
epochs = 1000


In [16]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/summary/{}'.format(tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tbCallBack])

# creates a HDF5 file 'my_model.h5'
model.save(file_name)

OSError: Unable to create file (unable to open file: name = '/Users/jaco/code/tabby/models/G30x30.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)